### 랭체인 도구
- OpenAI 의 function calling 과 tools 기능과 유사
- @tool 데코레이터를 사용하여 함수를 도구로 변환 ➡ 도구 등록 ➡ 언어 모델이 함수 호출하여 응답 생성m

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage
llm = ChatOpenAI(model="gpt-4.1-mini")

llm.invoke([HumanMessage("잘 지냈어?")])

AIMessage(content='네, 잘 지냈어요! 당신은 어떻게 지내셨어요? 도움이 필요하신 게 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 24, 'prompt_tokens': 12, 'total_tokens': 36, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEJKYlattyS46UY94cVv5VwZUHa8', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--ee33669c-743b-4fb0-ad05-d618e031567d-0', usage_metadata={'input_tokens': 12, 'output_tokens': 24, 'total_tokens': 36, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

In [2]:
from langchain_core.tools import tool
from datetime import datetime
import pytz

@tool # (1)tool 데코레이터를 사용하여 함수를 도구로 등록. docstring 또는 description 속성 필수
def get_current_time(timezone: str, location: str) -> str:
    """ 현재 시각을 반환하는 함수
    Args:
        timezone (str): 타임존 (예: 'Asia/Seoul') 실제 존재하는 타임존이어야 함
        location (str): 지역명. llm 이 전달해 주는 지역명 확인.
    """
    tz = pytz.timezone(timezone)
    now = datetime.now(tz).strftime("%Y-%m-%d %H:%M:%S")
    location_and_local_time = f'{timezone} ({location}) 현재시각 {now} ' # 타임존, 지역명, 현재시각을 문자열로 반환
    print(location_and_local_time)
    return location_and_local_time

In [3]:
# (2) 도구를 tools 리스트에 추가하고, tool_dict에도 추가
tools = [get_current_time,]
tool_dict = {"get_current_time": get_current_time,}

# (3)도구를 모델에 바인딩: 모델에 도구를 바인딩하면, 도구를 사용하여 llm 답변을 생성할 수 있음
llm_with_tools = llm.bind_tools(tools)

In [4]:
from langchain_core.messages import SystemMessage

# (4) 사용자의 질문과 tools 사용하여 llm 답변 생성
messages = [
    SystemMessage(content="너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다."),
    HumanMessage(content="도쿄는 지금 몇시야?"),     # response 에 tool_calls 항목 있음.
    # HumanMessage(content="AI 설명해줘"),    # response 에 tool_calls 항목 빈 리스트.
]

# (5) llm_with_tools를 사용하여 사용자의 질문에 대한 llm 답변 생성
response = llm_with_tools.invoke(messages)
messages.append(response)

# (6) 지금까지의 messages 확인
for i,msg in enumerate(messages):
  print(f'messages[{i}] : {msg}')

messages[0] : content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.' additional_kwargs={} response_metadata={}
messages[1] : content='도쿄는 지금 몇시야?' additional_kwargs={} response_metadata={}
messages[2] : content='' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 124, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEJL8rMwoJ4jcdfF6q6A1hpNkIjl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None} id='lc_run--59282b94-f208-43d8-b4bd-c78ac7e506ef-0' tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}, 'id': 'call_3cB0qKqGTpLxChkqaFNTHazE', 'type': 'tool_call'}] usag

In [5]:
print(response.tool_calls)    # messages[2].tool_calls[0] 과 같음

[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}, 'id': 'call_3cB0qKqGTpLxChkqaFNTHazE', 'type': 'tool_call'}]


In [6]:
# (7) response.tool_calls 있으면 저장된 함수 호출 정보를 하나씩 가져와 실행하고 결과는 ToolMessage 타입으로
#     messages 에 추가
# 
from langchain.messages import ToolMessage
for tool_call in response.tool_calls:
     # 중요: tool_call 구조 분해
    tool_name = tool_call.get("name")  # tool_call["name"] 문자열 리턴
    tool_args = tool_call.get("args", {})
    tool_call_id = tool_call.get("id")  # ← 이 ID가 매칭 키
    # tool_dict를 사용하여 도구 함수객체 리턴
    selected_tool = tool_dict[tool_name]  # tool_dict를 사용하여 도구 함수객체 리턴
    tool_result = selected_tool.invoke(tool_args)   # globals() 대신에 랭체인에서 함수 실행하기
    print(f'log args ➡ {tool_call["args"]}') # 도구 호출 시 전달된 인자 출력
    tool_result = selected_tool.invoke(tool_call)
    tool_message = ToolMessage(
                tool_call_id=tool_call_id,      # ← 도구 호출 ID와 매칭
                name=tool_name,                  # ← 도구 이름
                content=str(tool_result)         # ← 도구 실행 결과 (문자열)
    )
    messages.append(tool_message)
messages

Asia/Tokyo (Tokyo) 현재시각 2025-11-10 14:00:57 
log args ➡ {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}
Asia/Tokyo (Tokyo) 현재시각 2025-11-10 14:00:57 


[SystemMessage(content='너는 사용자의 질문에 답변을 하기 위해 tools를 사용할 수 있다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='도쿄는 지금 몇시야?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 124, 'total_tokens': 146, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4.1-mini-2025-04-14', 'system_fingerprint': 'fp_4c2851f862', 'id': 'chatcmpl-CaEJL8rMwoJ4jcdfF6q6A1hpNkIjl', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--59282b94-f208-43d8-b4bd-c78ac7e506ef-0', tool_calls=[{'name': 'get_current_time', 'args': {'timezone': 'Asia/Tokyo', 'location': 'Tokyo'}, 'id': 'call_3cB0qKqGTpLxChkqaFNTHazE', 'type': 'tool_ca

In [ ]:
messages[-1].content   # -> llm 에 전달해서 내용을 만들어 줘야 합니다.

"content='Asia/Tokyo (Tokyo) 현재시각 2025-11-10 14:00:57 ' name='get_current_time' tool_call_id='call_3cB0qKqGTpLxChkqaFNTHazE'"

In [8]:
response = llm_with_tools.invoke(messages)
print(f'💬 AI : {response.content}')

💬 AI : 도쿄는 현재 2025년 11월 10일 오후 2시 0분 57초입니다.
